In [21]:
import numpy as np
import pandas as pd

In [22]:
creadits = pd.read_csv('tmdb_5000_credits.csv')
movies = pd.read_csv('tmdb_5000_movies.csv')


In [71]:
# creadits.head()
movies.head()

In [23]:
movies= movies.merge(creadits, on='title')

In [26]:
movies= movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [32]:
movies.isnull().sum()

0

In [28]:
movies.dropna(inplace=True)

In [33]:
movies.duplicated().sum()


0

In [ ]:
# now we need to append the data of different collumns to create tags

In [44]:
# function to convert the string of list of objects of a column to a list of name in each object.
import ast
def fetchNamesList(list_string):
  L=[]
  for i in ast.literal_eval(list_string):
    L.append(i['name'])
  return L

def fetchTop3CastNames(list_string):
  L=[]
  count=0
  for i in ast.literal_eval(list_string):
    if count<3:
      L.append(i['name'])
      count+=1
    else: return L
  return L

def fetchDirector(list_string):
  L=[]
  for i in ast.literal_eval(list_string):
    if i['job']=="Director":
      L.append(i['name'])
      break
    
  return L

In [46]:
# filter all the columns to get only the required values

movies['genres']= movies['genres'].apply(fetchNamesList)
movies['keywords']= movies['keywords'].apply(fetchNamesList)
movies['cast']= movies['cast'].apply(fetchTop3CastNames)
movies['crew']= movies['crew'].apply(fetchDirector)

In [49]:
# remove space between words of names

movies['genres']= movies['genres'].apply(lambda x: [i.replace(" ", "") for i in x ])
movies['keywords']= movies['keywords'].apply(lambda x: [i.replace(" ", "") for i in x ])
movies['cast']= movies['cast'].apply(lambda x: [i.replace(" ", "") for i in x ])
movies['crew']= movies['crew'].apply(lambda x: [i.replace(" ", "") for i in x ])

In [53]:
# merging all the columns to create a single column called tags
movies['tags']= movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [57]:
new_movies= movies[['movie_id', 'title', 'tags']]

In [60]:
new_movies['tags']= new_movies['tags'].apply(lambda x: " ".join(x))

C:\Users\HP\AppData\Local\Temp\ipykernel_15640\1695453769.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies['tags']= new_movies['tags'].apply(lambda x: " ".join(x))


In [ ]:
# removing similar workds like action and actions 
# for that we use this posterstremmer that will make all the similar words to their root
# like making actions-> action

In [80]:
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()

In [81]:
def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [85]:
new_movies['tags']= new_movies['tags'].apply(stem)


'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

In [ ]:
# convert the tags into vectors so that we can plot it on axis and 
# plot them to find nearest of their neighbours

In [86]:
from sklearn.feature_extraction.text import CountVectorizer
cv= CountVectorizer(max_features=5000, stop_words="english")

In [89]:
vectors= cv.fit_transform(new_movies['tags']).toarray()

In [ ]:
# now finding the difference between any two movies in data frame of movies

In [90]:
from sklearn.metrics.pairwise import cosine_similarity

In [93]:
similarities= cosine_similarity(vectors)

In [100]:
def recomment(movie):
  movie_index= new_movies[new_movies['title']==movie].index[0]
  distance= similarities[movie_index]
  movie_list = sorted(list(enumerate(distance)), reverse=True, key= lambda x: x[1])[1:6]

  for i in movie_list:
    print(new_movies.iloc[i[0]]['title'])

In [101]:
recomment('Avatar')

Aliens vs Predator: Requiem
Falcon Rising
Independence Day
Titan A.E.
Aliens
